<a href="https://colab.research.google.com/github/Sid-2004/Research-Paper--WallStreetsBets-ML-Emotional-Sentiment/blob/main/Research_Paper_FinTech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install praw yfinance transformers torch tensorflow keras-tcn

In [ ]:
# ---------------------------
# 0. Setup: installs & imports
# ---------------------------
!pip install praw transformers sentence-transformers yfinance pytorch-lightning pytorch-forecasting==0.10.4 torchinfo
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu  # CPU fallback if GPU not available

# Standard imports
import os, re, time, math, json, datetime
from collections import Counter, defaultdict
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import networkx as nx
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import yfinance as yf


In [ ]:
!pip install praw yfinance transformers torch tensorflow keras-tcn
import praw
import pandas as pd

In [ ]:
reddit = praw.Reddit(client_id="gpXjQ2OtfofKUYW-rcMJrw",
client_secret="TfeCSLEoqahvSnikIFDYXdAavE7qww",
user_agent="my_wsb_scraper/0.1 by your_momsbob_is_mine"
)
subreddit = reddit.subreddit("wallstreetbets")

# Example: Get top 100 posts from this week
posts = []
for submission in subreddit.top(limit=100, time_filter="week"):
    posts.append({
        "title": submission.title,
        "text": submission.selftext,
        "score": submission.score,
        "comments": submission.num_comments
    })

In [ ]:
df = pd.DataFrame(posts)
df.head()

In [ ]:
df.tail()

In [ ]:
df.to_csv("wsb_posts.csv",index=False)
df.head()

In [ ]:
import re
df=pd.read_csv("wsb_posts.csv")

def clean_text(text):
  if not isinstance(text, str):
    return ""
  text = text.lower()                           # lowercase
  text = re.sub(r"http\S+|www\S+", "", text)    # remove links
  text = re.sub(r"[^a-z\s]", "", text)          # remove punctuation/numbers
  text = re.sub(r"\s+", " ", text).strip()      # remove extra spaces
  return text

df["clean_text"] = df["title"].astype(str) + " " + df["text"].astype(str)
df["clean_text"] = df["clean_text"].apply(clean_text)

display(df.head())

In [ ]:
# ---------------------------
# 1. Data Collection (using PRAW)
# ---------------------------
import praw
import pandas as pd

# Replace with your own Reddit API credentials (from https://www.reddit.com/prefs/apps)
# Make sure to keep these secret!
reddit = praw.Reddit(client_id="gpXjQ2OtfofKUYW-rcMJrw",
                     client_secret="TfeCSLEoqahvSnikIFDYXdAavE7qww",
                     user_agent="my_wsb_scraper/0.1 by your_momsbob_is_mine")

subreddit = reddit.subreddit("wallstreetbets")

# Get a larger number of top posts from this week using PRAW
posts = []
# Increase the limit to get more posts, up to 1000
for submission in subreddit.top(limit=1000, time_filter="week"):
    posts.append({
        "title": submission.title,
        "text": submission.selftext,
        "score": submission.score,
        "comments": submission.num_comments
    })

df = pd.DataFrame(posts)
df.to_csv("wsb_posts_large.csv", index=False)
print(f"Saved {len(df)} posts to wsb_posts_large.csv")
display(df.head())

**Reasoning**:
Initialize the PRAW Reddit object and retrieve a smaller number of top posts from the specified subreddit, then store the relevant data in a DataFrame.



In [ ]:
reddit = praw.Reddit(client_id="gpXjQ2OtfofKUYW-rcMJrw",
client_secret="TfeCSLEoqahvSnikIFDYXdAavE7qww",
user_agent="my_wsb_scraper/0.1 by your_momsbob_is_mine"
)
subreddit = reddit.subreddit("wallstreetbets")

# Get a smaller number of top posts from this week
posts = []
for submission in subreddit.top(limit=50, time_filter="week"):
    posts.append({
        "title": submission.title,
        "text": submission.selftext,
        "score": submission.score,
        "comments": submission.num_comments
    })

df = pd.DataFrame(posts)
display(df.head())

**Reasoning**:
Define the clean_text function, concatenate the title and text columns, apply the cleaning function, and display the head of the dataframe to verify the results.



In [ ]:
import re

def clean_text(text):
  if not isinstance(text, str):
    return ""
  text = text.lower()                           # lowercase
  text = re.sub(r"http\S+|www\S+", "", text)    # remove links
  text = re.sub(r"[^a-z\s]", "", text)          # remove punctuation/numbers
  text = re.sub(r"\s+", " ", text).strip()      # remove extra spaces
  return text

df["clean_text"] = df["title"].astype(str) + " " + df["text"].astype(str)
df["clean_text"] = df["clean_text"].apply(clean_text)

display(df.head())

## Analyze data

### Subtask:
Perform some basic analysis on the collected data.


In [ ]:
#calculating and display the average score and number of comments and find the highest score and most comments
average_score = df['score'].mean()
average_comments = df['comments'].mean()

print(f"Average Score: {average_score:.2f}")
print(f"Average Comments: {average_comments:.2f}")

highest_score_post = df.loc[df['score'].idxmax()]
most_comments_post = df.loc[df['comments'].idxmax()]

print("\nPost with Highest Score:")
display(highest_score_post)

print("\nPost with Most Comments:")
display(most_comments_post)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Word cloud
text = " ".join(df["clean_text"].tolist())
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(text)

plt.figure(figsize=(10,6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

# Post length distribution
df["length"] = df["clean_text"].apply(len)
df["length"].hist(bins=30, figsize=(8,4))
plt.title("Distribution of Post Lengths")
plt.show()


In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

# Apply VADER
df["vader_score"] = df["clean_text"].apply(lambda x: sia.polarity_scores(x)["compound"])
df["vader_sentiment"] = df["vader_score"].apply(
    lambda x: "positive" if x > 0.05 else ("negative" if x < -0.05 else "neutral")
)

df["vader_sentiment"].value_counts()


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Load FinBERT
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
finbert_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Apply FinBERT sentiment analysis
def get_finbert_sentiment(text):
    if not text.strip():
        return "neutral"
    result = finbert_pipeline(text[:512])[0]  # truncate long posts
    return result["label"]

df["finbert_sentiment"] = df["clean_text"].apply(get_finbert_sentiment)
df["finbert_sentiment"].value_counts()


In [ ]:
import matplotlib.pyplot as plt

comparison = pd.DataFrame({
    "VADER": df["vader_sentiment"].value_counts(),
    "FinBERT": df["finbert_sentiment"].value_counts()
}).fillna(0)

comparison.plot(kind="bar", figsize=(8,5))
plt.title("Sentiment Distribution: VADER vs FinBERT")
plt.show()

In [ ]:
df[["clean_text", "vader_sentiment", "finbert_sentiment"]].head(20)


In [ ]:
import matplotlib.pyplot as plt

comparison = pd.DataFrame({
    "VADER": df["vader_sentiment"].value_counts(),
    "FinBERT": df["finbert_sentiment"].value_counts()
}).fillna(0)

comparison.plot(kind="bar", figsize=(8,5))
plt.title("Sentiment Distribution: VADER vs FinBERT")
plt.show()

comparison


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

# Create sentiment pipeline
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Example text
result = nlp("The stock price is expected to rise after earnings.")
print(result)

In [ ]:
import pandas as pd

# Assuming your dataframe is df with a column 'cleaned_text'
results = df["clean_text"].apply(lambda x: nlp(x)[0]['label'])

df["finbert_sentiment"] = results
print(df["finbert_sentiment"].value_counts())

In [ ]:
!pip install transformers torch --quiet
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

In [ ]:
# Model trained on 124M tweets
MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment"

# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

# Build pipeline
twitter_roberta = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


In [ ]:
#SAMPLE TEXT RUN
example = "TWITTER is going to the moon 🚀🚀🚀"
print(twitter_roberta(example))

In [ ]:
example = "GME is going to DOWN ,WHAT TO DO"
print(twitter_roberta(example))


In [ ]:
# Apply sentiment analysis on cleaned text column
df["twitter_roberta_sentiment"] = df["clean_text"].apply(lambda x: twitter_roberta(x)[0]['label'])

# Count results
print(df["twitter_roberta_sentiment"].value_counts())

In [ ]:
print(df["twitter_roberta_sentiment"].value_counts())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot the distribution of Twitter RoBERTa sentiment labels
plt.figure(figsize=(6,4))
sns.countplot(x="twitter_roberta_sentiment", data=df, palette="viridis")
plt.title("Distribution of Twitter RoBERTa Sentiment Labels")
plt.xlabel("Sentiment Label")
plt.ylabel("Count")
plt.show()